# Example plots for visualizing persistent homology output

This notebook contains example visualizations that can be used to answer questions about the filtered complex's persistent homology. For example, these functions can be used to show

- persistence diagrams
- barcodes
- interactive barcodes with generators


We will generate fake data here for sake of the examples.

In [1]:
pip install ..

Processing /home/jovyan
  Created wheel for aptviz: filename=aptviz-0.0.1-py3-none-any.whl size=7812 sha256=1e5f03f5b3bb9c35812c1e4dfe18d7f61f058a9a50b7d13fedd7a5618215046d
  Stored in directory: /tmp/pip-ephem-wheel-cache-blo4gp9m/wheels/fc/c4/49/78b5bd16ca276f2916d0829d47c131046b6e4575f7dd51e987
Successfully built aptviz
  Attempting uninstall: aptviz
    Found existing installation: aptviz 0.0.1
    Uninstalling aptviz-0.0.1:
      Successfully uninstalled aptviz-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Load packages 

# General packages
import numpy as np
import sys
import pandas as pd

# Plotly
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# ApTViz
from aptviz import aptviz_themes
from aptviz.supporting import *
from aptviz.viz import *

# Load ApTViz visualization theme
pio.templates.default = "aptviz"

### Construct data

The first cell creates a fake simplicial complex intended for demonstrating the visualiation functions. Consider modifying the parameters `n_nodes`, `n_simps`, `max_dim`, and adding any indicator column you may find useful (such as `is_maximal`).

The second cell generates a fake barcode from a filtered simplicial complex and a parameter `n_bars` specifying the number of bars to generate. Bar dimension, birth, death, and representative simplices will be chosen at random.

In [3]:
## Generate data

# Parameters
n_nodes = 100
n_simps = [n_nodes, 150, 110, 90, 40, 30, 20, 15]
max_dim = int(len(n_simps)-1)
indicator_col = "is_maximal"

# Construct data frame
fsc_df = create_fake_fsc_df(n_nodes, n_simps, max_dim)

# Add indicator column
fsc_df = add_fake_indicator(fsc_df, indicator_col)

fsc_df.head()

Created df with length 555. Expected 555.


,cell_id,dim,nodes,weight,faces,rank,is_maximal
0,0,0,[0],7.115579,[],3,1
1,1,0,[1],7.893902,[],23,0
2,2,0,[2],7.467068,[],98,0
3,3,0,[3],7.982615,[],58,1
4,4,0,[4],7.455165,[],19,1


In [4]:
## Generate barcode

n_bars = 100
filter_on = "weight"

bar_df = create_fake_barcode(fsc_df, n_bars, filter_on)
bar_df.head()

,bar_id,bar_dim,bar_birth,bar_death,rep,lifetime
0,0,1,6.792394,5.857690,"[178, 192, 187, 249]",0.934705
1,1,5,2.827757,1.481210,"[497, 515, 506, 507]",1.346546
2,2,2,5.653710,4.756081,"[341, 313, 257, 324]",0.897629
3,3,2,5.734144,4.314789,"[280, 272, 304, 292]",1.419355
4,4,5,2.006781,1.392422,"[519, 516, 497, 493]",0.614359


## Persistence diagram (overlayed)

The following creates a standard persistence diagram with dimensions overlayed. The `plot_pd` function takes the barcode as well as an `axis_range` parameter (=`[max, min]` or `[min, max]`) that controls the x and y ranges. If bar birth and death are based on weight, ensure that the `axis_range[0] > axis_range[1]`, and the opposite for rank.

Depending on the number of bars, it may be helpful to adjust the marker size. The function has the marker size set to 12, but one can change this property (for example, to 10) with
`fig.update_traces(marker_size = 10)` .

In [10]:
# fig = plot_pd(bar_df, axis_range)

max_filtration = np.max(fsc_df[filter_on])
min_filtration = np.min(fsc_df[filter_on])

# Optional buffer -- prevents dots from being cut off in the figure
buffer = 0.5

# Plot! Be sure to note the input to axis range -- currently based on filter_on = "weight" (see above)
fig = plot_pd(bar_df, [max_filtration + buffer, min_filtration - buffer])

fig.show()

## Persistence diagram (faceted)

Alternatively, we can follow the small multiples concept and create many persistence diagram subplots -- one for each dimension. The `plot_pd_faceted` function works similar to the `plot_pd` function, except there is an optional argument `col_wrap` that controls the number of columns.

Consider modifying the size of the plot after generation as shown below.

In [33]:
# fig = plot_pd_faceted(bar_df, axis_range, col_wrap = 3)

# Keeping the axis_range the same as above.
fig = plot_pd_faceted(bar_df, [max_filtration + buffer, min_filtration - buffer])

fig.update_layout(height = 600,
                  width = 800)


fig.show()

In [7]:
fig = go.Figure()
bar_df_sorted = bar_df.sort_values(by=["bar_dim", "bar_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)

for b,bar in bar_df_sorted.iterrows():
    
    if (bar_df_sorted.iloc[b-1]["bar_dim"] != bar["bar_dim"]) :
        showlegendtf = True
    else :
        showlegendtf = False
        
    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines',
                    name=f'dim {bar["bar_dim"]}',
                    showlegend=showlegendtf,
                    line=dict(color=aptviz_themes.davos_colors[int(bar["bar_dim"])]),
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br><b>rep</b>: {bar["rep"]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()




In [8]:
### Color by a node involved in something or some sort of indicator column

bar_df["indicator"] = [np.random.randint(0, 2) for i in np.arange(bar_df.shape[0])]
bar_df.head()

on_off_colors = ["#CC7C33", "#3F4142"]
on_off_opacity = [1, 0.2]

fig = go.Figure()
bar_df_sorted = bar_df.sort_values(by=["bar_dim", "bar_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)

counter = 0
for b,bar in bar_df_sorted.iterrows():
    
    if (bar_df_sorted.iloc[b-1]["indicator"] != bar["indicator"]) & (counter < 2) :
        showlegendtf = True
        counter = counter +1
        print(counter)
    else :
        showlegendtf = False


    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines',
                    name=f'indicator {bar["indicator"]}',
                    opacity = on_off_opacity[bar["indicator"]],
                    showlegend=showlegendtf,
                    line=dict(color=on_off_colors[int(bar["indicator"])]),
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br><b>rep</b>: {bar["rep"]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()

1
2


In [9]:
### Color by numeric value - such as persistence or number of nodes/simplices in generator


col_of_interest = "indicator"

bar_df[col_of_interest] = [np.random.rand() for i in np.arange(bar_df.shape[0])]
vmin = np.min(bar_df[col_of_interest])
vmax = np.max(bar_df[col_of_interest])

norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = matplotlib.cm.get_cmap('cividis')


fig = go.Figure()
bar_df_sorted = bar_df.sort_values(by=["bar_dim", "bar_birth"])
bar_df_sorted = bar_df_sorted.reset_index(drop=True)


for b,bar in bar_df_sorted.iterrows():
    
    if (b==1) :
        showlegendtf = True
    else :
        showlegendtf = False
        

    fig.add_trace(go.Scatter(x=[bar["bar_birth"], bar["bar_death"]], y=[b+1, b+1],
                    mode='lines+markers',
                    marker = dict(size=0.1,
                                  showscale = showlegendtf,
                                  colorscale='cividis',
                                  cmin=0,
                                  cmax=50,
                                  colorbar = dict(title=col_of_interest)),
                    name=f'bar {bar["bar_id"]}',
                    showlegend=False,
                    line=dict(color=f'rgba{cmap(norm(bar[col_of_interest]))}'),
                    hovertemplate =
                        f'<i>id</i>: {bar["bar_id"]}'+
                        f'<br>rep: {bar["rep"]}'+
                        f'<br><b>{col_of_interest}</b>: {bar[col_of_interest]}'))



# Add nice effects and such
fig.update_layout(title_text="Barcode")
fig.update_yaxes(title_text = "H_k")
fig.update_xaxes(title_text = "Filtration index")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout(hovermode="x unified")
fig.show()


NameError: name 'matplotlib' is not defined